In [ ]:
import numpy as np
import pandas as pd
import random

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score as r2, accuracy_score
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LinearRegression

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

from matplotlib import rcParams
# matplotlib.rcParams.update({'font.size': 10})
pd.options.display.max_columns = 30

In [ ]:
import warnings
warnings.filterwarnings('ignore')

**Описание датасета**

* **Id** - идентификационный номер квартиры
* **DistrictId** - идентификационный номер района
* **Rooms** - количество комнат
* **Square** - площадь
* **LifeSquare** - жилая площадь
* **KitchenSquare** - площадь кухни
* **Floor** - этаж
* **HouseFloor** - количество этажей в доме
* **HouseYear** - год постройки дома
* **Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности
* **Social_1, Social_2, Social_3** - социальные показатели местности
* **Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья
* **Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров
* **Price** - цена квартиры

In [ ]:
train = pd.read_csv('train.csv')
train.head(3)

In [ ]:
test = pd.read_csv('test.csv')
test.head(3)

In [ ]:
train.shape

In [ ]:
test.shape

#### Приведение типов данных

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
train['Id'].dtype

In [ ]:
test['Id'].dtype

In [ ]:
train['Id'] = train['Id'].astype(str)
train['Id'].dtype

In [ ]:
test['Id'] = test['Id'].astype(str)
test['Id'].dtype

In [ ]:
train.describe()

In [ ]:
test.describe()

Что не нравится и хотелось бы скорректировать:

1. приведение категориальных признаков Ecology_2, Ecology_3, Shops_2 в бинарные
* некорректные значения HouseYear в train
* недостаток данных Healthcare_1
* недостаток данных LifeSquare
* max KitchenSquare выглядит слишком большим в обоих датасетах
* max LifeSquare выглядит слишком большим в train
* min LifeSquare выглядит слишком малым в обоих датасетах

Что касается площадей - в моем понимании Square > KitchenSquare + LifeSquare. Исходя из этой логики в дальнейшем будем пытаться скорректировать значения в датасетах

#### 1. Приведение категориальных признаков Ecology_2, Ecology_3, Shops_2 в бинарные
Проверим количество возможных значений в категориальных признаках Ecology_2, Ecology_3, Shops_2 датасета train:

In [ ]:
print(train['Ecology_2'].value_counts()) 
print(train['Ecology_3'].value_counts()) 
print(train['Shops_2'].value_counts())

Т.к. каждый из категориальных признаков Ecology_2, Ecology_3, Shops_2 содержит только два варианта значений (A или B), то мы можем оставить только одно из них, которое будет соответствовать '1', а '0' будет соответствовать другому варианту. Заметим также, что в этих признаках нет пропусков.

In [ ]:
train = pd.concat([train, 
                   pd.get_dummies(pd.DataFrame({'Ecology_2':train['Ecology_2'], 
                                                'Ecology_3':train['Ecology_3'], 
                                                'Shops_2':train['Shops_2']}), 
                                  prefix=['Ecol_2', 'Ecol_3', 'Shops_2'], 
                                  prefix_sep='_', 
                                  drop_first=True, 
                                  dtype='int64')
                  ], axis=1)
train.drop(['Ecology_2', 'Ecology_3', 'Shops_2'], axis=1, inplace=True)
train.head()

In [ ]:
train.shape

Аналогично проверим количество возможных значений в категориальных признаках Ecology_2, Ecology_3, Shops_2 датасета test, а также убедимся в отсутствии пропусков, и проведем те же манипуляции над датасетом для приведения названных признаков в бинарные:

In [ ]:
print(test['Ecology_2'].value_counts()) 
print(test['Ecology_3'].value_counts()) 
print(test['Shops_2'].value_counts())

In [ ]:
test = pd.concat([test, 
                   pd.get_dummies(pd.DataFrame({'Ecology_2':test['Ecology_2'], 
                                                'Ecology_3':test['Ecology_3'], 
                                                'Shops_2':test['Shops_2']}), 
                                  prefix=['Ecol_2', 'Ecol_3', 'Shops_2'], 
                                  prefix_sep='_', 
                                  drop_first=True, 
                                  dtype='int64')
                  ], axis=1)
test.drop(['Ecology_2', 'Ecology_3', 'Shops_2'], axis=1, inplace=True)
test.head()

In [ ]:
test.shape

#### 2. Исправим значение HouseYear в датасете train

In [ ]:
train.loc[train['HouseYear'] > 2020]

Из предыдущего вывода видно 2 выброса c DistrictId 109 и 147. Т.к. это всего два значения, они не должны оказать сильного влияния на обучение, но, тем не менее являются выбросами. Предположу замену HouseYear в первом случае на 2011, а во втором на 1968.

In [ ]:
train.loc[(train['HouseYear'] == 20052011), 'HouseYear'] = 2011
train.loc[(train['HouseYear'] == 4968), 'HouseYear'] = 1968
pd.concat([train.loc[1497:1497,:], train.loc[4189:4189,:]], axis=0)

#### 3. Заполним недостающие данные Healthcare_1

In [ ]:
# рассмотрим таблицу корреляций признака Healthcare_1
train.corr()[['Healthcare_1']]

Исходя из предыдущей таблицы и сведений о признаках предположу, что признак Healthcare_1 зависит от (в порядке убывания):
 - DistrictId
 - Social_1
 - Social_2
 - Social_3
 - Helthcare_2

Заполняем пропуски Healthcare_1

In [ ]:
def ident_model(train_df):
    model_1 = RandomForestClassifier(max_depth=29)
    train_ = train_df[train_df['Healthcare_1'].isna() == False][['DistrictId', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2','Healthcare_1']]
    tr, tst = train_test_split(train_, test_size = 0.15, random_state=29)
    model_1.fit(tr.drop('Healthcare_1', axis = 1), tr['Healthcare_1'])
    pred = model_1.predict(tst.drop('Healthcare_1', axis = 1))
    print(accuracy_score(tst['Healthcare_1'], pred))
    test_ = train_df[train_df['Healthcare_1'].isna()][['DistrictId', 'Social_1', 'Social_2', 'Social_3', 'Helthcare_2','Healthcare_1']]
    model_1.fit(train_.drop('Healthcare_1', axis = 1), train_['Healthcare_1'])
    pred = model_1.predict(test_.drop('Healthcare_1', axis = 1))
    return model_1


def fill_H1(model_1, train_df):
    test_ = train_df[train_df['Healthcare_1'].isna()][['DistrictId', 'Social_1','Social_2', 'Social_3', 'Helthcare_2','Healthcare_1']]
    pred = model_1.predict(test_.drop('Healthcare_1', axis = 1))
    return pred

In [ ]:
model_1 = ident_model(train)
pred = fill_H1(model_1, train)

In [ ]:
# заполняем пропуски в тренировочном датасете
train.loc[train['Healthcare_1'].isna(), 'Healthcare_1'] = pred
train.isna().sum()

In [ ]:
# заполняем тестовый датасет
predict_test_H1 = fill_H1(model_1, test)
test.loc[test['Healthcare_1'].isna(), 'Healthcare_1'] = predict_test_H1
test.isna().sum()

#### 4. Заполним недостающие данные LifeSquare

In [ ]:
# заполняем LifeSquare как (Square - KitchenSquare) * (медиану отношения LifeSquare/(Square - KitchenSquare) для Square > LifeSquare
def fill_LifeSquare(train_df):
    train_df.loc[(train_df['Square'] < train_df['LifeSquare']) | (train_df['LifeSquare'].isna()), 'LifeSquare'] = \
    (train_df.loc[(train_df['Square'] < train_df['LifeSquare']) | (train_df['LifeSquare'].isna()), 'Square'] - \
    train_df.loc[(train_df['Square'] < train_df['LifeSquare']) | (train_df['LifeSquare'].isna()), 'KitchenSquare']) * \
    (train_df.loc[(train_df['Square'] > train_df['LifeSquare']), 'LifeSquare'] / \
    (train_df.loc[(train_df['Square'] > train_df['LifeSquare']), 'Square'] - \
    train_df.loc[(train_df['Square'] > train_df['LifeSquare']), 'KitchenSquare'])).median()
    return train_df

In [ ]:
# заполняем пропуски в тренировочном датасете
train = fill_LifeSquare(train)
train.isna().sum()

In [ ]:
# аналогично для тестового датасета
test = fill_LifeSquare(test)
test.isna().sum()

In [ ]:
# заменяем HouseFloor
def fill_HouseFloor(df):
    df.loc[df['Floor'] > df['HouseFloor'], 'HouseFloor'] = df.loc[df['Floor'] > df['HouseFloor'], 'Floor']
    return df

train = fill_HouseFloor(train)
test = fill_HouseFloor(test)

In [ ]:
#заполняем комнаты
def fill_rooms(df):
    # df.loc[(df['Rooms'] > 5) | (df['Rooms'] == 0), 'Rooms'] = 5
    df.loc[(df['Rooms'] > 5), 'Rooms'] = 5
    df.loc[(df['Rooms'] == 0), 'Rooms'] = 1
    return df

train = fill_rooms(train)
test = fill_rooms(test)

#### Создаем новый признак, средней цены на кв. метр по району

#### 5. Прогнозирование

### Кросс-валидация

### Кластеризация: k-means и последующее выделение 3-х кластеров

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def display_clusters_distribution(unique_labels, labels_counts):
    plt.figure(figsize=(8,5))

    plt.bar(unique, counts)

    plt.xlabel('Clusters')
    plt.xticks(unique)
    plt.ylabel('Count')
    plt.title('Clusters distribution')
    plt.show()

In [ ]:
kmeans_3 = KMeans(n_clusters=3, random_state=29)
labels_clast_3 = kmeans_3.fit_predict(train.drop(['Id', 'Price'], axis = 1))
labels_clast_3 = pd.Series(labels_clast_3, name='clusters_3')

unique, counts = np.unique(labels_clast_3, return_counts=True)
display_clusters_distribution(unique, counts)

### Добавление новых признаков

In [ ]:
train_ext = pd.concat([train, labels_clast_3], axis=1)
train_ext.sort_values(by=['clusters_3', 'Id'], ascending=True, inplace=True)
train_ext

### Кластеризация тестового датасета и добавление в него новых признаков

In [ ]:
labels_clast_3_test = kmeans_3.predict(test.drop(['Id'], axis = 1))
labels_clast_3_test = pd.Series(labels_clast_3_test, name='clusters_3')

unique, counts = np.unique(labels_clast_3_test, return_counts=True)
display_clusters_distribution(unique, counts)

In [ ]:
test_ext = pd.concat([test, labels_clast_3_test], axis=1)
test_ext.sort_values(by=['clusters_3', 'Id'], ascending=True, inplace=True)
test_ext

Max valid R2 = 0.7463025606261264 ( train R2 = 0.9473107664191112 )
max_depth = 15

Max valid R2 = 0.5509655649093166 ( train R2 = 0.9290883513385566 )
max_depth = 11

Max valid R2 = 0.7656705315953981 ( train R2 = 0.9479899543634576 )
max_depth = 12

### Кросс-валидация

In [ ]:
parameters = {
    'n_estimators': [100, 150, 200, 250, 300, 350, 400],
    'max_features': np.arange(9, 19),
    'max_depth': np.arange(11, 21),
}

In [ ]:
rgr0 = GridSearchCV(
    estimator=RandomForestRegressor(random_state=29),
    param_grid=parameters,
    scoring='r2',
    cv=5,
)

In [ ]:
# Со всеми признаками
# X_train, X_test, y_train, y_test = train_test_split(train.drop(['Id','Price'], axis = 1), 
#                                                     train['Price'], test_size = 0.16, random_state=29)
clst_3_num = 0 # номер кластера
X_train, X_test, y_train, y_test = train_test_split(train_ext[train_ext['clusters_3'] == clst_3_num].
                                                    drop(['Id', 'Price', 'clusters_3'], axis = 1), 
                                                    train_ext.loc[train_ext['clusters_3'] == clst_3_num, 'Price'], 
                                                    test_size = 0.2, random_state=29)

rgr0.fit(X_train, y_train)

In [ ]:
rgr0.best_params_

In [ ]:
rgr1 = GridSearchCV(
    estimator=RandomForestRegressor(random_state=29),
    param_grid=parameters,
    scoring='r2',
    cv=5,
)

In [ ]:
# Со всеми признаками
# X_train, X_test, y_train, y_test = train_test_split(train.drop(['Id','Price'], axis = 1), 
#                                                     train['Price'], test_size = 0.16, random_state=29)
clst_3_num = 1 # номер кластера
X_train, X_test, y_train, y_test = train_test_split(train_ext[train_ext['clusters_3'] == clst_3_num].
                                                    drop(['Id', 'Price', 'clusters_3'], axis = 1), 
                                                    train_ext.loc[train_ext['clusters_3'] == clst_3_num, 'Price'], 
                                                    test_size = 0.2, random_state=29)

rgr1.fit(X_train, y_train)

In [ ]:
rgr1.best_params_

In [ ]:
rgr2 = GridSearchCV(
    estimator=RandomForestRegressor(random_state=29),
    param_grid=parameters,
    scoring='r2',
    cv=5,
)

In [ ]:
# Со всеми признаками
# X_train, X_test, y_train, y_test = train_test_split(train.drop(['Id','Price'], axis = 1), 
#                                                     train['Price'], test_size = 0.16, random_state=29)
clst_3_num = 2 # номер кластера
X_train, X_test, y_train, y_test = train_test_split(train_ext[train_ext['clusters_3'] == clst_3_num].
                                                    drop(['Id', 'Price', 'clusters_3'], axis = 1), 
                                                    train_ext.loc[train_ext['clusters_3'] == clst_3_num, 'Price'], 
                                                    test_size = 0.2, random_state=29)

rgr2.fit(X_train, y_train)

In [ ]:
rgr2.best_params_

Полученная в результате модель аналогична такой модели:

In [ ]:
rgr = RandomForestRegressor(max_depth=18, max_features=5, n_estimators=320, random_state=29)

rgr.fit(X_train, y_train)

y_pred = rgr.predict(X_test)

r2(y_test, y_pred)

### Обучение модели кластерно на всех данных и итоговое предсказание с найденными `best_params_`

In [ ]:
rgr_0 = RandomForestRegressor(max_depth=18, max_features=5, n_estimators=320, random_state=29)
rgr_0.fit(train_ext[train_ext['clusters_3'] == 0].drop(['Id', 'Price', 'clusters_3'], axis = 1) , 
          train_ext.loc[train_ext['clusters_3'] == 0, 'Price'])

In [ ]:
rgr_1 = RandomForestRegressor(max_depth=18, max_features=5, n_estimators=320, random_state=29)
rgr_1.fit(train_ext[train_ext['clusters_3'] == 1].drop(['Id', 'Price', 'clusters_3'], axis = 1) , 
          train_ext.loc[train_ext['clusters_3'] == 1, 'Price'])

In [ ]:
rgr_2 = RandomForestRegressor(max_depth=18, max_features=5, n_estimators=320, random_state=29)
rgr_2.fit(train_ext[train_ext['clusters_3'] == 2].drop(['Id', 'Price', 'clusters_3'], axis = 1) , 
          train_ext.loc[train_ext['clusters_3'] == 2, 'Price'])

#### Делаем предсказание

In [ ]:
itog_predict_rfr_CV_kmeans_full_model_c0 = rgr_0.predict(test_ext[(test_ext['clusters_3'] == 0)].drop(['Id', 'clusters_3'], axis = 1))

In [ ]:
itog_predict_rfr_CV_kmeans_full_model_c1 = rgr_1.predict(test_ext[(test_ext['clusters_3'] == 1)].drop(['Id', 'clusters_3'], axis = 1))

In [ ]:
itog_predict_rfr_CV_kmeans_full_model_c2 = rgr_2.predict(test_ext[(test_ext['clusters_3'] == 2)].drop(['Id', 'clusters_3'], axis = 1))

In [ ]:
itog_predict_rfr_CV_kmeans_cls_full_model = np.hstack([itog_predict_rfr_CV_kmeans_full_model_c0, 
                                                       itog_predict_rfr_CV_kmeans_full_model_c1, 
                                                       itog_predict_rfr_CV_kmeans_full_model_c2])

In [ ]:
test_ext['Price'] = itog_predict_rfr_CV_kmeans_cls_full_model

In [ ]:
test_ext[['Id', 'Price']].to_csv('rfr_CV_kmeans_cls_full_model_rev07.csv', index = False)

In [ ]:
imp = pd.DataFrame()
imp['name'] = test_ext.drop(['Id','Price', 'clusters_3'], axis = 1).columns
imp['value'] = rgr_0.feature_importances_
print(imp.sort_values('value' , ascending = False))

In [ ]:
imp = pd.DataFrame()
imp['name'] = test_ext.drop(['Id','Price', 'clusters_3'], axis = 1).columns
imp['value'] = rgr_1.feature_importances_
print(imp.sort_values('value' , ascending = False))

In [ ]:
imp = pd.DataFrame()
imp['name'] = test_ext.drop(['Id','Price', 'clusters_3'], axis = 1).columns
imp['value'] = rgr_2.feature_importances_
print(imp.sort_values('value' , ascending = False))

### Сохранение модели

**Scaler**

**Model**